# CNN
Classificação de Imagens de acordo com dados de CIFAR10
Classes : ['Avião', 'Automóvel','Passáro','Gato','Cervo','Cachorro', 'Sapo', 'Cavalo', 'Navio', 'Caminhão']

In [1]:
# Importando Bibliotecas
import os
import torch
import matplotlib.pyplot as plt
from torchvision.datasets import CIFAR10

In [2]:
# Variveis Globais
DATA_DIR = './'
CATEGORIES = ['Avião', 'Automóvel','Passáro','Gato','Cervo','Cachorro', 'Sapo', 'Cavalo', 'Navio', 'Caminhão']

In [3]:
# Dados
cifar10_train = CIFAR10(DATA_DIR, train=True, download=True)
cifar10_test = CIFAR10(DATA_DIR, train=False, download=True)

100%|██████████| 170498071/170498071 [00:13<00:00, 12857172.24it/s]


Extracting ./cifar-10-python.tar.gz to ./
Files already downloaded and verified


In [12]:
# Quantidade de dados para treinamento e teste
(len(cifar10_train), len(cifar10_test))

(50000, 10000)

In [5]:
import torchvision.transforms as tt
prep_transform = tt.Compose([tt.ToTensor(), tt.Normalize(
    (0.4914, 0.4822, 0.4465),
    (0.2470, 0.2435, 0.2616)
)])
tensor_train = CIFAR10(DATA_DIR, train=True, download=False, transform=prep_transform)
tensor_test = CIFAR10(DATA_DIR, train=False, download=False, transform=prep_transform)


In [6]:
imgs = torch.stack([img_t for img_t, _ in tensor_train], dim=3)
imgs.shape

torch.Size([3, 32, 32, 50000])

In [7]:
imgs.view(3, -1).mean(dim=1) # Média das Cores

tensor([-1.2762e-06, -1.7074e-04,  1.1819e-04])

In [8]:
imgs.view(3, -1).std(dim=1) # Desvio Padrão das Cores

tensor([1.0001, 0.9999, 1.0000])

In [9]:
from torch.utils.data import DataLoader
batch_size = 64

train_loader = DataLoader(tensor_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(tensor_test, batch_size=batch_size, shuffle=True)

In [15]:
# Testando o MLP para classificar imagem

from torch import nn

class MLPClassifier(nn.Module) :
    def __init__(self) :
        super().__init__()

        self.flatten = nn.Flatten()

        self.layers = nn.Sequential(
            nn.Linear(32 * 32 * 3, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128,64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )

    def forward(self, x) :
        v = self.flatten(x)
        return self.layers(v)

In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'Rodando na {device}')

Rodando na cuda


In [16]:
model = MLPClassifier().to(device)

In [ ]:
optimizer = torch.optim.SGD(model.parameter(), lr=0.001)